 # Daily Profit for Trades in TradeBench

In [89]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys  # Only needed to determine Python version
import matplotlib  # Only needed to determine Matplotlib version number
import chardet
from datetime import date
from pandas import DataFrame
from scipy import stats
import seaborn as sns

# Enable inline plotting
# %matplotlib inline

print('Python Version: ' + sys.version)
print('Pandas Version: ' + pd.__version__)
print('Matplotlib Version: ' + matplotlib.__version__)

Python Version: 3.7.6 (default, Jan  8 2020, 13:42:34) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Pandas Version: 1.0.1
Matplotlib Version: 3.1.3


In [90]:
#/Users/<USERNAME>/<CSV_LOCATION>.csv
CSV_Location = r'/Users/Pho/Documents/Dev/DayTrading/TradingAnalysisScripts/TradeBenchCSV/2020-09-17-TradeBench-closed-trades-export.csv'

df = pd.read_csv(CSV_Location, encoding = "ISO-8859-1")

# del df["Post Trade Analysis"]

print(df.columns)

Index(['Symbol', 'Name', 'Long/Short', 'Plan Date', 'Entry (first) Date',
       'Entry (first) Time', 'Exit (last) Date', 'Exit (last) Time',
       'Quantity', 'Avg. Entry', 'Avg. Exit', 'Entry Commission',
       'Exit Commission', 'Net P/L Amount', 'Net P/L %', 'Broker', 'Strategy',
       'Planned Stop Loss Price', 'First Stop Loss Price',
       'Trade Plan Description', 'Post Trade Analysis', 'Label 1', 'Label 2',
       'Label 3', 'Label 4', 'Label 5'],
      dtype='object')


In [91]:
# Combine dates and times together
df["Entry Date"] = df["Entry (first) Date"].astype(str) + " " + df["Entry (first) Time"].astype(str)
df["Exit Date"] = df["Exit (last) Date"].astype(str) + " " + df["Exit (last) Time"].astype(str)

df["Entry Date"] = df["Entry Date"].astype(str)
#print(df["Entry Date"])

Location2 = f'/Users/Pho/Documents/Dev/DayTrading/TradingAnalysisScripts/TradeBenchCSV/ting2.csv'
df.to_csv(Location2)

df["Entry Date"] = pd.to_datetime(df["Entry Date"], format="%d/%m/%Y %H:%M:%S")
df["Exit Date"] = pd.to_datetime(df["Exit Date"], format="%d/%m/%Y %H:%M:%S")

#print(df.info(verbose = True))
print(df["Entry Date"])

0    2020-08-10 14:37:00
1    2020-08-10 15:26:00
2    2020-08-11 14:41:00
3    2020-08-11 14:46:00
4    2020-08-11 14:54:00
             ...        
69   2020-09-15 16:01:00
70   2020-09-16 14:43:00
71   2020-09-16 14:48:00
72   2020-09-16 15:13:00
73   2020-09-16 15:22:00
Name: Entry Date, Length: 74, dtype: datetime64[ns]


In [92]:
#Group by hour, day, month live paper trades only
#NOTE in order to get this to work, you might have to conver the columns to date type in excel.

df_Live_PaperTrades = df[df["Broker"] == "PaperTrade Broker"]

byHour = df_Live_PaperTrades.groupby(df_Live_PaperTrades["Entry Date"].dt.hour)["Net P/L Amount"].agg([np.ma.count, np.sum, np.mean, np.median, stats.mode, np.ptp, stats.iqr, np.std, "min", "max"])

byDay = df_Live_PaperTrades.groupby(df_Live_PaperTrades["Entry Date"].dt.date)["Net P/L Amount"].agg([np.ma.count, np.sum, np.mean, np.median, stats.mode, np.ptp, stats.iqr, np.std, "min", "max"])

byMonth = df_Live_PaperTrades.groupby(df_Live_PaperTrades["Entry Date"].dt.month)["Net P/L Amount"].agg([np.ma.count, np.sum, np.mean, np.median, stats.mode, np.ptp, stats.iqr, np.std, "min", "max"])

print(byDay)

count    sum       mean  median             mode    ptp      iqr  \
Entry Date                                                                     
2020-08-10    2.0  -6.92  -3.460000  -3.460  ([-20.55], [1])  34.18  17.0900   
2020-08-11    4.0  16.21   4.052500   4.790    ([0.62], [1])   5.39   1.8725   
2020-08-12    4.0  23.69   5.922500   8.370  ([-27.09], [1])  61.13  28.3625   
2020-08-14    1.0   1.05   1.050000   1.050    ([1.05], [1])   0.00   0.0000   
2020-08-17    4.0 -26.93  -6.732500  -0.765   ([-25.7], [1])  26.00   7.1825   
2020-08-18    5.0  20.35   4.070000  -2.950  ([-28.39], [1])  73.84  55.0600   
2020-08-24    1.0  53.22  53.220000  53.220   ([53.22], [1])   0.00   0.0000   
2020-08-25    2.0  43.11  21.555000  21.555    ([0.41], [1])  42.29  21.1450   
2020-08-26    3.0 -53.85 -17.950000 -25.390  ([-25.51], [1])  22.56  11.2800   
2020-08-28    4.0  -7.64  -1.910000  -6.090  ([-25.75], [1])  56.04  43.3200   
2020-09-01    1.0  45.66  45.660000  45.660   ([45.6

In [93]:
#Summary
TotalSum = df_Live_PaperTrades["Net P/L Amount"].sum()
TotalWinSum = df_Live_PaperTrades[df_Live_PaperTrades["Net P/L Amount"] < 0].groupby("date")
Mean = df_Live_PaperTrades["Net P/L Amount"].mean()
print(TotalWinSum)

KeyError: 'date'